In [ ]:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [1]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langgraph

Name: langgraph
Version: 0.0.66
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core
Required-by: 


# The goal

The input is a customer email.

1. Categorize the email
2. Write a draft email response

The ouput is an is a response to the customer email.

In [3]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [4]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

# Utils

In [6]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


# Basic chains

1. Categorize email
2. Write draft email

In [7]:
#Categorize EMAIL
# Categorizes the email into one of these categories:
# 'price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic'

# Using PromptTemplate allows for
# different variables to be
# input into the prompt.
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Email Categorizer Agent You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        price_equiry - used when someone is asking for information about pricing \
        customer_complaint - used when someone is complaining about something \
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
        customer_feedback - used when someone is giving feedback about a product \
        off_topic when it doesnt relate to any other category \


            Output a single cetgory only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

# Create the chain using
# LangChain expression language
email_category_generator = prompt | GROQ_LLM | StrOutputParser()



# Example

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)


'customer_feedback'


In [8]:
## Write Draft Email

draft_writer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent take the INITIAL_EMAIL below  from a human that has emailed our company email address, the email_category \
            that the categorizer agent gave it and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information that hasn't been provided by the research_info or in the initial_email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

draft_writer_chain = draft_writer_prompt | GROQ_LLM | JsonOutputParser()


# Example

email_category = 'customer_feedback'

print(draft_writer_chain.invoke({"initial_email": EMAIL, "email_category":email_category}))


{'email_draft': "Dear Paul,\n\nThank you so much for taking the time to share your wonderful experience at our resort! We're thrilled to hear that you had a great stay and that our staff made a positive impact on your visit. Your kind words truly mean the world to us, and we're grateful for customers like you who make our hard work worthwhile.\n\nWe'll make sure to pass on your appreciation to our staff, as it's feedback like yours that motivates them to continue providing exceptional service.\n\nOnce again, thank you for choosing to stay with us, and we hope to have the pleasure of welcoming you back soon!\n\nBest regards,\nSarah, Resident Manager"}


# Tool setup

No tools are used.

# State

In [12]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [13]:
from typing_extensions import TypedDict
from typing import List


# The state must contain all variables that go into nodes and
# come out of nodes. This includes the input to the graph and output
# from the graph.

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_email: email
        email_category: email category
        draft_email: LLM generation
        final_email: LLM generation
        research_info: list of documents
        info_needed: whether to add search info
        num_steps: number of steps
    """
    initial_email : str
    email_category : str
    draft_email : str
    final_email : str
    research_info : List[str]
    info_needed : bool
    num_steps : int
    draft_email_feedback : dict

# Nodes

1. categorize_email
2. draft_email_writer

In [ ]:
# 1. Each output from a function must correspond to a variable in the "state".
# 2. It looks like the "state" object gets created automatically - we just define the class.
# 3. The state gets updated automatically as the flow exits each node.
# 3. When we don't want to update the state after completing a node,
# we return nothing - as in the state printer below.

In [14]:
def categorize_email(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)
    # save to local disk
    write_markdown_file(email_category, "email_category")

    return {"email_category": email_category, "num_steps":num_steps}

In [15]:
def draft_email_writer(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = draft_writer_chain.invoke({"initial_email": initial_email,
                                     "email_category": email_category})

    print(draft_email)
    # print(type(draft_email))

    email_draft = draft_email['email_draft']
    write_markdown_file(email_draft, "draft_email")

    return {"draft_email": email_draft, "num_steps":num_steps}

In [18]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state['initial_email']} \n" )
    print(f"Email Category: {state['email_category']} \n")
    print(f"Draft Email: {state['draft_email']} \n" )
    print(f"Final Email: {state['final_email']} \n" )
    print(f"Research Info: {state['research_info']} \n")
    print(f"Info Needed: {state['info_needed']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

# Conditional Edges

None

# Build the Graph

## Add nodes

In [22]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_email", categorize_email)
workflow.add_node("draft_email_writer", draft_email_writer)
workflow.add_node("state_printer", state_printer)

## Add edges

In [23]:
# Here we say which node connects to which node.
# An edge is an arrow in the graph (including the "input" and "output" arrows)
# In essence, we are defining every arrow in the graph.

# Input
workflow.set_entry_point("categorize_email")

workflow.add_edge("categorize_email", "draft_email_writer")

workflow.add_edge("draft_email_writer", "state_printer")

# Output
workflow.add_edge("state_printer", END)


# Compile
app = workflow.compile()

# Run the system

In [27]:
EMAIL = """HI there, \n
I am emailing to say that I was disappointed with the food during my recent stay.

I really hope you fix this next time.

Thanks,
George
"""


inputs = {"initial_email": EMAIL, "num_steps":0}

output = app.invoke(inputs)

---CATEGORIZING INITIAL EMAIL---
'customer_complaint'
---DRAFT EMAIL WRITER---
{'email_draft': "Dear George,\n\nThank you for taking the time to share your feedback about your recent stay with us. I'm so sorry to hear that the food didn't meet your expectations. I want to assure you that we value your opinion and are taking your comments seriously. We're committed to providing the best possible experience for our guests, and we'll do our best to address the issues you've raised.\n\nCould you please tell me more about your experience with the food during your stay? What specifically didn't meet your expectations? This will help us to better understand what went wrong and make the necessary improvements.\n\nThank you again for your feedback. We appreciate your honesty and look forward to the opportunity to serve you better in the future.\n\nBest regards,\nSarah, Resident Manager"}
---STATE PRINTER---
Initial Email: HI there, 

I am emailing to say that I was disappointed with the food du

In [25]:
!ls

draft_email.md	email_category.md  sample_data
